# Import

In [13]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm


# Fixed RandomSeed & Setting Hyperparameter

In [14]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [15]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Load

In [16]:
train = pd.read_csv("./dataset/train/train.csv")

# Define Model

In [17]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

# Train

In [18]:
def train_lstm(train_df):
    trained_models = {}

    for (store_menu,), group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models

In [19]:
# 학습
trained_models = train_lstm(train)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]

Training LSTM: 100%|██████████| 193/193 [01:49<00:00,  1.77it/s]


# Prediction

In [20]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for (store_menu,), store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        recent_vals_df = pd.DataFrame(recent_vals, columns=['매출수량'])
        recent_vals_scaled = scaler.transform(recent_vals_df)
        x_input = torch.tensor([recent_vals_scaled]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        pred_scaled_reshaped = pred_scaled.reshape(-1, 1)
        restored_pred = scaler.inverse_transform(pred_scaled_reshaped)
        restored = np.round(np.maximum(0, restored_pred)).astype(int).flatten()

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })


    return pd.DataFrame(results)


In [21]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./dataset/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)
    
full_pred_df = pd.concat(all_preds, ignore_index=True)

# Submission

In [22]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [23]:
sample_submission = pd.read_csv('./dataset/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('./results/baseline_submission.csv', index=False, encoding='utf-8-sig')
submission

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,6,19,5,2,0,1,10,2,2,...,9,8,27,9,48,19,7,23,7,14
1,TEST_00+2일,6,18,5,3,1,1,11,2,2,...,8,12,24,11,41,17,7,24,7,17
2,TEST_00+3일,6,16,5,3,0,1,8,1,2,...,9,13,23,11,29,19,9,31,7,18
3,TEST_00+4일,6,22,5,3,0,1,11,2,2,...,8,13,23,13,52,21,9,28,7,14
4,TEST_00+5일,5,15,5,2,1,1,10,2,2,...,9,10,25,11,52,20,8,29,7,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,5,25,5,3,0,1,8,1,2,...,10,14,25,13,63,22,34,64,18,33
66,TEST_09+4일,5,31,5,3,0,1,12,1,2,...,9,14,24,14,84,24,35,61,18,28
67,TEST_09+5일,4,26,5,2,1,1,11,1,2,...,10,11,26,13,87,23,33,59,18,31
68,TEST_09+6일,4,27,5,2,1,1,8,1,2,...,10,11,29,13,74,23,33,56,18,29
